### Only supports EVM chains for now.

In [57]:
from web3 import Web3
from dotenv import load_dotenv
import os
import json
import requests
import json
from datetime import datetime
# Load environment variables from .env file
load_dotenv()


web3 = Web3(Web3.HTTPProvider(os.getenv('ALCHEMY_ETHEREUM_RPC_URL')))

with open('abis/ERC20.json', 'r') as f:
    ERC20_ABI = json.load(f)

In [58]:
def is_contract_address(address: str) -> bool:
    """
    Check if an address is a contract address
    
    Args:
        address: Ethereum address to check
        
    Returns:
        bool: True if contract address, False if EOA (externally owned account)
    """
    # First verify it's a valid address
    if not web3.isAddress(address):
        return False
    
    # Get the code at the address
    code = web3.eth.get_code(web3.toChecksumAddress(address))
    
    # If there's code at the address, it's a contract
    # If no code (b'0x' or empty bytes), it's an EOA
    return code != b'' and code != b'0x'

In [60]:
def get_bitquery_access_token() -> str:
    url = "https://oauth2.bitquery.io/oauth2/token"

    payload = f'grant_type=client_credentials&client_id={os.getenv("BITQUERY_CLIENT_ID")}&client_secret={os.getenv("BITQUERY_CLIENT_SECRET")}&scope=api'

    headers = {'Content-Type': 'application/x-www-form-urlencoded'}

    response = requests.request("POST", url, headers=headers, data=payload)
    resp = json.loads(response.text)

    return resp['access_token']

access_token = get_bitquery_access_token()

# Headers with your authorization token
def get_token_holders(token_address: str, limit: int = 10) -> dict:
    """
    Get top token holders for a given token address using BitQuery API
    
    Args:
        token_address: Ethereum token contract address
        limit: Number of top holders to return (default 10)
        
    Returns:
        dict: Response containing token holder data
    """

    url = "https://streaming.bitquery.io/graphql"
    headers = {
        'Content-Type': 'application/json',
        'Authorization': f"Bearer {access_token}"
    }

    payload = json.dumps({
        "query": """
        {
      EVM(network: eth, dataset: archive) {
        TokenHolders(
          date: "%s"
          tokenSmartContract: "%s"
          limit: {count: %d}
          orderBy: {descendingByField: "Balance_Amount"}
        ) {
          Balance {
            Amount
          }
          Holder {
            Address
          }
        }
      }
    }
        """ % (datetime.now().strftime("%Y-%m-%d"), token_address, limit)
    })

    try:
        response = requests.post(url, headers=headers, data=payload)
        holders = response.json()['data']['EVM']['TokenHolders']
        return [{'balance': float(h['Balance']['Amount']), 'address': h['Holder']['Address']} for h in holders]
    except Exception as e:
        print(f"An error occurred: {str(e)}")
        return None
    
get_token_holders("0x95aD61b0a150d79219dCF64E1E6Cc01f0B64C4cE", 20)

[{'balance': 410429558672705.2,
  'address': '0xdead000000000000000042069420694206942069'},
 {'balance': 44734539692849.46,
  'address': '0x02e2201576fbbefb52812f2ee7f08eb4774b481e'},
 {'balance': 44196872316250.12,
  'address': '0xf977814e90da44bfa03b6295a0616a897441acec'},
 {'balance': 39273245781062.02,
  'address': '0x40b38765696e3d5d8d9d834d8aad4bb6e418e489'},
 {'balance': 33306962858144.39,
  'address': '0xa023f08c70a23abc7edfc5b6b5e171d78dfc947e'},
 {'balance': 9039646468451.545,
  'address': '0x73af3bcf944a6559933396c1577b257e2054d935'},
 {'balance': 8940000000000.0,
  'address': '0x47ac0fb4f2d84898e4d9e7b4dab3c24507a6d503'},
 {'balance': 8830090712624.2,
  'address': '0x6cc5f688a315f3dc28a7781717a9a798a59fda7b'},
 {'balance': 8294714935467.388,
  'address': '0xa7060aece919e4b7d9928b8418a9d30731471fa9'},
 {'balance': 6956614520659.673,
  'address': '0x422193843fe209faa94f5cc1780e04965e77cb7f'},
 {'balance': 6684282197912.395,
  'address': '0x5a52e96bacdabb82fd05763e25335261b270

In [72]:
def assert_token_address(token_address: str):
    assert web3.isAddress(token_address), "Invalid token address"

def monitor_whale_wallets(token_address: str):
    assert_token_address(token_address)
    # Create contract instance
    token_contract = web3.eth.contract(address=token_address, abi=ERC20_ABI)
    
    # Call totalSupply method
    decimals = token_contract.functions.decimals().call()
    total_supply = token_contract.functions.totalSupply().call() / (10 ** decimals)
    
    # Define whale threshold ( >0.5% of supply)
    MIN_WHALE_HOLDINGS = total_supply * 0.005

    # Get all token holders
    holders = get_token_holders(token_address, 25)
    
    # Filter holders by balance
    holders_with_balance = [holder for holder in holders if holder['balance'] > MIN_WHALE_HOLDINGS and not is_contract_address(holder['address'])]

    return holders_with_balance
    
    

data = monitor_whale_wallets("0x95aD61b0a150d79219dCF64E1E6Cc01f0B64C4cE")
data

[{'balance': 410429558672705.2,
  'address': '0xdead000000000000000042069420694206942069'},
 {'balance': 44734539692849.46,
  'address': '0x02e2201576fbbefb52812f2ee7f08eb4774b481e'},
 {'balance': 44196872316250.12,
  'address': '0xf977814e90da44bfa03b6295a0616a897441acec'},
 {'balance': 39273245781062.02,
  'address': '0x40b38765696e3d5d8d9d834d8aad4bb6e418e489'},
 {'balance': 33306962858144.39,
  'address': '0xa023f08c70a23abc7edfc5b6b5e171d78dfc947e'},
 {'balance': 9039646468451.545,
  'address': '0x73af3bcf944a6559933396c1577b257e2054d935'},
 {'balance': 8940000000000.0,
  'address': '0x47ac0fb4f2d84898e4d9e7b4dab3c24507a6d503'},
 {'balance': 8830090712624.2,
  'address': '0x6cc5f688a315f3dc28a7781717a9a798a59fda7b'},
 {'balance': 8294714935467.388,
  'address': '0xa7060aece919e4b7d9928b8418a9d30731471fa9'},
 {'balance': 6684282197912.395,
  'address': '0x5a52e96bacdabb82fd05763e25335261b270efcb'},
 {'balance': 5775767675523.355,
  'address': '0xcffad3200574698b78f32232aa9d63eabd29